Перебрати всіх дітей і порівняти з попереднім результатом, щоб знайти нових

In [2]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [211]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import json
import tqdm
import time


In [212]:
'''
    find last li in pagination. Neet to known the last page in childlist
'''
url = "https://www.msp.gov.ua/children/search.php?form=*&male=*&age_from=*&age_to=*&region=*&brothers=*&needs=*&page=1"
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    paragraphs = soup.find_all("ul", class_="pagination") #find pagination block
    for i, paragraph in enumerate(paragraphs, start=1):
        li_elements = paragraph.find_all("li") #find all li in pagination
        
        if li_elements:
            last_li = li_elements[-1] #find al li in pagination block
            last_a = last_li.text.strip() #get text last li
            last_page = int(re.sub(r'\D','',last_a)) #leve only page number    
else:
    print(f"ERROR {response.status_code}")


In [ ]:
'''
    This script is time-consuming. Therefore, it should only be run when you want to update data for all children.
    
    get array of all children on page
'''
all_children = []
for i in tqdm.tqdm(range(last_page)): #(last_page):
    url = "https://www.msp.gov.ua/children/search.php?form=*&male=*&age_from=*&age_to=*&region=*&brothers=*&needs=*&page="+str(i+1)
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        child_block = soup.find_all("div",class_="child__item")
        for i, child in enumerate(child_block, start=1):
            link = child.find("a")["href"]
            id_child = re.search(r'n=(\d+)', link)
            if id_child:
                id_child = id_child.group(1)
            else:
                print(f"В картці дитини на сторінці {url}, не знайдено id code")
            #print(id_child,"\n-------------------")
            all_children.append(id_child)
    else:
        print(f"ERROR {response.status_code}")

json_output = json.dumps(all_children, indent=4)
#print(json_output)

with open('all_children.json', 'r') as json_file:
    data = json.load(json_file)

differences = list(set(all_children) ^ set(data))
print(differences)

with open('all_children.json', 'w') as json_file:
    json_file.write(json_output)  # Записуємо JSON в файл

if len(differences)>0:
    with open('differences.json', 'w') as json_file:
        json_file.write(json.dumps(differences, indent=4))  # Записуємо JSON в файл
